# Statistical Tests for Accuracy Differences

Run McNemar's test to see if accuracy differences are significant.

In [25]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

In [26]:
# Dataframe to store all test results
results = pd.DataFrame(columns=['Model A', 'Model B', 'p, Non-Exact', 'p, Non-Exact, Continuity Corrected', 'p, Exact'])

## Function

In [27]:
def run_test(merged_data, model_1_name, model_2_name):
    '''
    Run McNemar's Test on two models' predictions.

    Parameters:
    - merged_data: a pandas DataFrame with columns 'ticker', 'fixed_quarter_date', and 'correct_model_1' and 'correct_model_2', which are boolean values indicating whether the prediction was correct for each model.
    '''

    print('model 1 accuracy: ' + str(merged_data['correct_model_1'].mean()))
    print('model 2 accuracy: ' + str(merged_data['correct_model_2'].mean()))

    # Fill contingency table cells
    m1_correct_m2_correct = merged_data[(merged_data['correct_model_1'] == True) & (merged_data['correct_model_2'] == True)].shape[0]
    m1_correct_m2_incorrect = merged_data[(merged_data['correct_model_1'] == True) & (merged_data['correct_model_2'] == False)].shape[0]
    m1_incorrect_m2_correct = merged_data[(merged_data['correct_model_1'] == False) & (merged_data['correct_model_2'] == True)].shape[0]
    m1_incorrect_m2_incorrect = merged_data[(merged_data['correct_model_1'] == False) & (merged_data['correct_model_2'] == False)].shape[0]
    
    # Create a contingency table
    # Rows for m1 correct and incorrect
    # Columns for m2 correct and incorrect
    contingency_table = [[m1_correct_m2_correct, m1_correct_m2_incorrect], 
                         [m1_incorrect_m2_correct, m1_incorrect_m2_incorrect]]
  
    print('McNemar\'s Test contingency table:')
    print(contingency_table)

    # McNemar's Test, non-exact, without any continuity correction 
    print('Test results, non-exact, no continuity correction:')
    print(mcnemar(contingency_table, exact=False, correction=False)) 
    ncc_p_value = mcnemar(contingency_table, exact=False, correction=False).pvalue

    # McNemar's Test, non-exact, with continuity correction
    print('Test results, non-exact, with continuity correction:')
    print(mcnemar(contingency_table, exact=False, correction=True))
    cc_p_value = mcnemar(contingency_table, exact=False, correction=True).pvalue

    # McNemar's Test, exact, without any continuity correction
    print('Test results, exact, no continuity correction:')
    print(mcnemar(contingency_table, exact=True, correction=False))
    ec_p_value = mcnemar(contingency_table, exact=True, correction=False).pvalue

    # Add results to the results dataframe
    results.loc[len(results)] = [model_1_name, model_2_name, ncc_p_value, cc_p_value, ec_p_value]

    # ncc_result = mcnemar(contingency_table, exact=True, correction=False)
    # ncc_statistic = ncc_result.statistic
    # ncc_p_value = ncc_result.pvalue

def test_models(model_1_short_path, model_2_short_path, model_1_name, model_2_name):

    print('running for ' + model_1_short_path + ' and ' + model_2_short_path)

    # Load first model from '../../Data/Predictions/' + model_1_short_path
    model_1_data = pd.read_excel('../../Data/Predictions/' + model_1_short_path)
    print(model_1_data.head())
    # Third column is true, last is predicted, add column for 'correct'
    model_1_data['correct'] = model_1_data.iloc[:, 2] == model_1_data.iloc[:, 3]
    # Load second model from '../../Data/Predictions/' + model_2_short_path
    model_2_data = pd.read_excel('../../Data/Predictions/' + model_2_short_path)
    print(model_2_data.head())
    # Third column is true, last is predicted, add column for 'correct'
    model_2_data['correct'] = model_2_data.iloc[:, 2] == model_2_data.iloc[:, 3]
    # Merge the two datasets on 'ticker' and 'fixed_quarter_date'
    merged_data = pd.merge(model_1_data, model_2_data, on=['ticker', 'fixed_quarter_date'], suffixes=('_model_1', '_model_2'))

    # Run test
    run_test(merged_data, model_1_name, model_2_name)

## Run Tests

### Logistic Versus Random Guess

In [28]:
test_models('Logistic Regression/exclude_previous_rating_model_1/exclude_previous_rating_model_1_predictions.xlsx',
            'Random Guess/Rating_RG_Preds.xlsx',
            'Logistic - Exclude Previous Rating, Altman-Z Only',
            'Rating Random Guess Model')

running for Logistic Regression/exclude_previous_rating_model_1/exclude_previous_rating_model_1_predictions.xlsx and Random Guess/Rating_RG_Preds.xlsx
  ticker fixed_quarter_date Rating exclude_previous_rating_model_1_predictions
0   AAPL         2016-07-01     AA                                         AAA
1   ABBV         2015-04-01      A                                          AA
2   ABBV         2016-04-01      A                                          BB
3    ABC         2012-04-01      A                                          BB
4    ABC         2013-01-01      A                                          BB
  ticker fixed_quarter_date Rating pred
0   AAPL         2016-07-01     AA   BB
1   AMZN         2013-07-01     AA  BBB
2    ATO         2016-10-01     AA  BBB
3    CAT         2016-04-01     AA   BB
4   CSCO         2016-04-01     AA    A
model 1 accuracy: 0.19230769230769232
model 2 accuracy: 0.22450805008944544
McNemar's Test contingency table:
[[47, 168], [204, 699]]
T

### NLP Features Contribution

In [29]:
test_models('Logistic Regression/exclude_previous_rating_model_2/exclude_previous_rating_model_2_predictions.xlsx',
            'Logistic Regression/exclude_previous_rating_model_3/exclude_previous_rating_model_3_predictions.xlsx',
            'Logistic - Exclude Previous Rating Model 2',
            'Logistic - Exclude Previous Rating Model 3')

running for Logistic Regression/exclude_previous_rating_model_2/exclude_previous_rating_model_2_predictions.xlsx and Logistic Regression/exclude_previous_rating_model_3/exclude_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date Rating exclude_previous_rating_model_2_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                          AA
2   ABBV         2016-04-01      A                                          AA
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                          AA
2   ABBV         2016-04-01      A                                     

In [30]:
test_models('Logistic Regression/include_previous_rating_model_2/include_previous_rating_model_2_predictions.xlsx',
            'Logistic Regression/include_previous_rating_model_3/include_previous_rating_model_3_predictions.xlsx', 
            'Logistic - Include Previous Rating Model 2', 
            'Logistic - Include Previous Rating Model 3')

running for Logistic Regression/include_previous_rating_model_2/include_previous_rating_model_2_predictions.xlsx and Logistic Regression/include_previous_rating_model_3/include_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date Rating include_previous_rating_model_2_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
  ticker fixed_quarter_date Rating include_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                     

c:\Users\ijyli\miniforge3\envs\capstone\Lib\site-packages\statsmodels\stats\contingency_tables.py:1348: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))
c:\Users\ijyli\miniforge3\envs\capstone\Lib\site-packages\statsmodels\stats\contingency_tables.py:1348: RuntimeWarning: divide by zero encountered in scalar divide
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))


In [31]:
test_models('Logistic Regression/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx',
            'Logistic Regression/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx',
            'Logistic - SMOTE Rating Change Model 2',
            'Logistic - SMOTE Rating Change Model 3')

running for Logistic Regression/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx and Logistic Regression/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fixed Quarter Date   
1   ABBV         2015-04-01                Same As Last Fixed Quarter Date   
2   ABBV         2016-04-01                Same As Last Fixed Quarter Date   
3    ABC         2012-04-01                Same As Last Fixed Quarter Date   
4    ABC         2013-01-01                Same As Last Fixed Quarter Date   

   smote_rating_change_model_2_predictions  
0          Same As Last Fixed Quarter Date  
1  Downgrade Since Last Fixed Quarter Date  
2          Same As Last Fixed Quarter Date  
3          Same As Last Fixed Quarter Date  
4          Same As Last Fixed Quarter Date  
  ticker fixed_quarter_date Change Direction Since Last Fix

In [32]:
test_models('XGBoost/exclude_previous_rating_model_2/exclude_previous_rating_model_2_predictions.xlsx',
            'XGBoost/exclude_previous_rating_model_3/exclude_previous_rating_model_3_predictions.xlsx',
            'XGBoost - Exclude Previous Rating Model 2',
            'XGBoost - Exclude Previous Rating Model 3')

running for XGBoost/exclude_previous_rating_model_2/exclude_previous_rating_model_2_predictions.xlsx and XGBoost/exclude_previous_rating_model_3/exclude_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date Rating exclude_previous_rating_model_2_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC         2012-04-01      A                                          BB
4    ABC         2013-01-01      A                                           A
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC        

In [33]:
test_models('XGBoost/include_previous_rating_model_2/include_previous_rating_model_2_predictions.xlsx',
            'XGBoost/include_previous_rating_model_3/include_previous_rating_model_3_predictions.xlsx',
            'XGBoost - Include Previous Rating Model 2',
            'XGBoost - Include Previous Rating Model 3')

running for XGBoost/include_previous_rating_model_2/include_previous_rating_model_2_predictions.xlsx and XGBoost/include_previous_rating_model_3/include_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date Rating include_previous_rating_model_2_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
  ticker fixed_quarter_date Rating include_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC        

c:\Users\ijyli\miniforge3\envs\capstone\Lib\site-packages\statsmodels\stats\contingency_tables.py:1348: RuntimeWarning: invalid value encountered in scalar divide
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))
c:\Users\ijyli\miniforge3\envs\capstone\Lib\site-packages\statsmodels\stats\contingency_tables.py:1348: RuntimeWarning: divide by zero encountered in scalar divide
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))


In [34]:
test_models('XGBoost/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx',
            'XGBoost/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx',
            'XGBoost - SMOTE Rating Change Model 2',
            'XGBoost - SMOTE Rating Change Model 3')

running for XGBoost/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx and XGBoost/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fixed Quarter Date   
1   ABBV         2015-04-01                Same As Last Fixed Quarter Date   
2   ABBV         2016-04-01                Same As Last Fixed Quarter Date   
3    ABC         2012-04-01                Same As Last Fixed Quarter Date   
4    ABC         2013-01-01                Same As Last Fixed Quarter Date   

  smote_rating_change_model_2_predictions  
0         Same As Last Fixed Quarter Date  
1         Same As Last Fixed Quarter Date  
2         Same As Last Fixed Quarter Date  
3         Same As Last Fixed Quarter Date  
4         Same As Last Fixed Quarter Date  
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL   

In [35]:
test_models('XGBoost/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx',
            'Random Guess/Change_RG_Preds.xlsx',
            'XGBoost - SMOTE Rating Change Model 2',
            'Change Random Guess Model')

running for XGBoost/smote_rating_change_model_2/smote_rating_change_model_2_predictions.xlsx and Random Guess/Change_RG_Preds.xlsx
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fixed Quarter Date   
1   ABBV         2015-04-01                Same As Last Fixed Quarter Date   
2   ABBV         2016-04-01                Same As Last Fixed Quarter Date   
3    ABC         2012-04-01                Same As Last Fixed Quarter Date   
4    ABC         2013-01-01                Same As Last Fixed Quarter Date   

  smote_rating_change_model_2_predictions  
0         Same As Last Fixed Quarter Date  
1         Same As Last Fixed Quarter Date  
2         Same As Last Fixed Quarter Date  
3         Same As Last Fixed Quarter Date  
4         Same As Last Fixed Quarter Date  
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fi

In [36]:
test_models('XGBoost/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx',
            'Random Guess/Change_RG_Preds.xlsx',
            'XGBoost - SMOTE Rating Change Model 3',
            'Change Random Guess Model')

running for XGBoost/smote_rating_change_model_3/smote_rating_change_model_3_predictions.xlsx and Random Guess/Change_RG_Preds.xlsx
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fixed Quarter Date   
1   ABBV         2015-04-01                Same As Last Fixed Quarter Date   
2   ABBV         2016-04-01                Same As Last Fixed Quarter Date   
3    ABC         2012-04-01                Same As Last Fixed Quarter Date   
4    ABC         2013-01-01                Same As Last Fixed Quarter Date   

  smote_rating_change_model_3_predictions  
0         Same As Last Fixed Quarter Date  
1         Same As Last Fixed Quarter Date  
2         Same As Last Fixed Quarter Date  
3         Same As Last Fixed Quarter Date  
4         Same As Last Fixed Quarter Date  
  ticker fixed_quarter_date Change Direction Since Last Fixed Quarter Date  \
0   AAPL         2016-07-01                Same As Last Fi

### Graph NN Tests

In [37]:
# Reformatted GNN data

# Example
inductive_gnn_test = pd.read_excel('../../Data/Predictions/Graph Neural Network/Inductive/exclude_previous_rating_model_2_predictions.xlsx')
print(inductive_gnn_test.head())

def reformat_gnn_data(short_path):
    # Load GNN data
    gnn_df = pd.read_excel('../../Data/Predictions/Graph Neural Network/' + short_path)
    # Split node on ' : ' into 'ticker' and 'fixed_quarter_date'
    gnn_df[['ticker', 'fixed_quarter_date']] = gnn_df['node'].str.split(' : ', expand=True)
    # Keep only 'ticker', 'fixed_quarter_date', 'target', pred
    gnn_df = gnn_df[['ticker', 'fixed_quarter_date', 'target', 'pred']]
    # Save to new file
    # New path - replace / with /reformatted_
    new_path = short_path.replace('/', '/reformatted_')
    gnn_df.to_excel('../../Data/Predictions/Graph Neural Network/' + new_path, index=False)

reformat_gnn_data('Transductive/exclude_previous_rating_model_2_predictions.xlsx')
reformat_gnn_data('Transductive/exclude_previous_rating_model_3_predictions.xlsx')
reformat_gnn_data('Inductive/exclude_previous_rating_model_2_predictions.xlsx')
reformat_gnn_data('Inductive/exclude_previous_rating_model_3_predictions.xlsx')

  target pred               node
0     AA   AA  AAPL : 2016-07-01
1      A    A  ABBV : 2015-04-01
2      A    A  ABBV : 2016-04-01
3      A    A   ABC : 2012-04-01
4      A  BBB   ABC : 2013-01-01


#### Transductive

In [38]:
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/lr_retrain_predictions.xlsx',
            'GNN - Transductive - Model 3',
            'Logistic Regression - Retrain on GNN Data')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/lr_retrain_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
  ticker fixed_quarter_date Rating prediction
0   AAPL         2016-07-01     AA         AA
1   ABBV         2015-04-01      A         AA
2   ABBV         2016-04-01      A         AA
3    ABC         2012-04-01      A          A
4    ABC         2013-01-01      A          A
model 1 accuracy: 0.676530612244898
model 2 accuracy: 0.639795918367347
McNemar's Test contingency table:
[[515, 148], [112, 205]]
Test results, non-exact, no continuity correction:
pvalue      0.025573669368214657
statistic   4.984615384615385
Test results, non-exact, with continuity correcti

In [39]:
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/lr_pretrained_predictions.xlsx',
            'GNN - Transductive - Model 3',
            'Logistic Regression - Pretrained on GNN Data')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/lr_pretrained_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                          AA
2   ABBV         2016-04-01      A                                         AAA
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
model 1 accuracy: 0.676530612244898
model 2 accuracy: 0.6469387755102041
McNemar's Test conti

In [40]:
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/xgb_retrain_predictions.xlsx',
            'GNN - Transductive - Model 3',
            'XGBoost - Retrain on GNN Data')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/xgb_retrain_predictions.xlsx


  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
  ticker fixed_quarter_date Rating prediction
0   AAPL         2016-07-01     AA         AA
1   ABBV         2015-04-01      A          A
2   ABBV         2016-04-01      A          A
3    ABC         2012-04-01      A          A
4    ABC         2013-01-01      A          A
model 1 accuracy: 0.676530612244898
model 2 accuracy: 0.9030612244897959
McNemar's Test contingency table:
[[639, 24], [246, 71]]
Test results, non-exact, no continuity correction:
pvalue      1.3561625012089296e-41
statistic   182.53333333333333
Test results, non-exact, with continuity correction:
pvalue      3.094095739224916e-41
statistic   180.8925925925926
Test results, exact, no continuity correction:
pvalue      1.478103668632942e-47
statistic   24.0


In [41]:
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/xgb_pretrained_predictions.xlsx',
            'GNN - Transductive - Model 3',
            'XGBoost - Pretrained on GNN Data')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/xgb_pretrained_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
model 1 accuracy: 0.676530612244898
model 2 accuracy: 0.9071428571428571
McNemar's Test cont

In [42]:
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx',
            'Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'GNN - Transductive - Model 2',
            'GNN - Transductive - Model 3')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx and Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
model 1 accuracy: 0.6632653061224489
model 2 accuracy: 0.676530612244898
McNemar's Test contingency table:
[[611, 39], [52, 278]]
Test results, non-exact, no continuity correction:
pvalue      0.17295491798841672
statistic   1.8571428571428572
Test results, non-exact, with continuity correction:
pvalue      0.208

#### Transductive versus Inductive

In [43]:
# Test models
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx',
            'Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx',
            'GNN - Transductive - Model 2',
            'GNN - Inductive - Model 2')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx and Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
model 1 accuracy: 0.6632653061224489
model 2 accuracy: 0.6653061224489796
McNemar's Test contingency table:
[[605, 45], [47, 283]]
Test results, non-exact, no continuity correction:
pvalue      0.8348273291852392
statistic   0.043478260869565216
Test results, non-exact, with continuity correction:
pvalue      0.9169

In [44]:
# Test models
test_models('Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'GNN - Transductive - Model 3',
            'GNN - Inductive - Model 3')

running for Graph Neural Network/Transductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A   AA
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A    A
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
model 1 accuracy: 0.676530612244898
model 2 accuracy: 0.6836734693877551
McNemar's Test contingency table:
[[617, 46], [53, 264]]
Test results, non-exact, no continuity correction:
pvalue      0.48172771631662636
statistic   0.494949494949495
Test results, non-exact, with continuity correction:
pvalue      0.5464935

#### Inductive

In [45]:
test_models('Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/lr_retrain_predictions.xlsx',
            'GNN - Inductive - Model 3',
            'Logistic Regression - Retrain on GNN Data')

running for Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/lr_retrain_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date Rating prediction
0   AAPL         2016-07-01     AA         AA
1   ABBV         2015-04-01      A         AA
2   ABBV         2016-04-01      A         AA
3    ABC         2012-04-01      A          A
4    ABC         2013-01-01      A          A
model 1 accuracy: 0.6836734693877551
model 2 accuracy: 0.639795918367347
McNemar's Test contingency table:
[[511, 159], [116, 194]]
Test results, non-exact, no continuity correction:
pvalue      0.00951434446647381
statistic   6.723636363636364
Test results, non-exact, with continuity correction:

In [46]:
test_models('Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/lr_pretrained_predictions.xlsx',
            'GNN - Inductive - Model 3',
            'Logistic Regression - Pretrained on GNN Data')

running for Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/lr_pretrained_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                          AA
2   ABBV         2016-04-01      A                                         AAA
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
model 1 accuracy: 0.6836734693877551
model 2 accuracy: 0.6469387755102041
McNemar's Test conting

In [47]:
test_models('Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/xgb_retrain_predictions.xlsx',
            'GNN - Inductive - Model 3',
            'XGBoost - Retrain on GNN Data')

running for Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/xgb_retrain_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date Rating prediction
0   AAPL         2016-07-01     AA         AA
1   ABBV         2015-04-01      A          A
2   ABBV         2016-04-01      A          A
3    ABC         2012-04-01      A          A
4    ABC         2013-01-01      A          A
model 1 accuracy: 0.6836734693877551
model 2 accuracy: 0.9030612244897959
McNemar's Test contingency table:
[[648, 22], [237, 73]]
Test results, non-exact, no continuity correction:
pvalue      1.0433221550712604e-40
statistic   178.47490347490347
Test results, non-exact, with continuity correct

In [48]:
test_models('Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'Graph Neural Network/Other Classifiers on GNN Data/xgb_pretrained_predictions.xlsx',
            'GNN - Inductive - Model 3',
            'XGBoost - Pretrained on GNN Data')

running for Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx and Graph Neural Network/Other Classifiers on GNN Data/xgb_pretrained_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date Rating exclude_previous_rating_model_3_predictions
0   AAPL         2016-07-01     AA                                          AA
1   ABBV         2015-04-01      A                                           A
2   ABBV         2016-04-01      A                                           A
3    ABC         2012-04-01      A                                           A
4    ABC         2013-01-01      A                                           A
model 1 accuracy: 0.6836734693877551
model 2 accuracy: 0.9071428571428571
McNemar's Test contin

In [49]:
# Test models
test_models('Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx',
            'Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx',
            'GNN - Inductive - Model 2',
            'GNN - Inductive - Model 3')

running for Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_2_predictions.xlsx and Graph Neural Network/Inductive/reformatted_exclude_previous_rating_model_3_predictions.xlsx
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
  ticker fixed_quarter_date target pred
0   AAPL         2016-07-01     AA   AA
1   ABBV         2015-04-01      A    A
2   ABBV         2016-04-01      A    A
3    ABC         2012-04-01      A    A
4    ABC         2013-01-01      A  BBB
model 1 accuracy: 0.6653061224489796
model 2 accuracy: 0.6836734693877551
McNemar's Test contingency table:
[[600, 52], [70, 258]]
Test results, non-exact, no continuity correction:
pvalue      0.1031768800930285
statistic   2.6557377049180326
Test results, non-exact, with continuity correction:
pvalue      0.123778009

## P-Values Table

In [50]:
p_values_table = results.copy()

# Round last 3 columns to two decimal places
p_values_table['p, Non-Exact'] = p_values_table['p, Non-Exact'].round(2)
p_values_table['p, Non-Exact, Continuity Corrected'] = p_values_table['p, Non-Exact, Continuity Corrected'].round(2)
p_values_table['p, Exact'] = p_values_table['p, Exact'].round(2)

# If p, Non-Exact is NaN, set all three columns to NaN
p_values_table.loc[p_values_table['p, Non-Exact'].isnull(), ['p, Non-Exact', 'p, Non-Exact, Continuity Corrected', 'p, Exact']] = None

# Convert all columns to strings and replace NaN with '-'
p_values_table = p_values_table.astype(str)
p_values_table = p_values_table.replace('nan', '-')

# In Model A and Model B, replace ' Model 2' with 'Fin Only'
p_values_table['Model A'] = p_values_table['Model A'].str.replace(' Model 2', ' Fin Only')
p_values_table['Model B'] = p_values_table['Model B'].str.replace(' Model 2', ' Fin Only')
# In Model A and Model B, replace ' Model 3' with 'Fin + NLP'
p_values_table['Model A'] = p_values_table['Model A'].str.replace(' Model 3', ' Fin + NLP')
p_values_table['Model B'] = p_values_table['Model B'].str.replace(' Model 3', ' Fin + NLP')
# Repalce 'Rating Fin' with 'Rating - Fin'
p_values_table['Model A'] = p_values_table['Model A'].str.replace('Rating Fin', 'Rating - Fin')
p_values_table['Model B'] = p_values_table['Model B'].str.replace('Rating Fin', 'Rating - Fin')
# Replace 'Change Fin' with 'Change - Fin'
p_values_table['Model A'] = p_values_table['Model A'].str.replace('Change Fin', 'Change - Fin')
p_values_table['Model B'] = p_values_table['Model B'].str.replace('Change Fin', 'Change - Fin')
# Replace 'on GNN Data' with 'on GNN Data - Fin + NLP'
p_values_table['Model A'] = p_values_table['Model A'].str.replace('on GNN Data', 'on GNN Data - Fin + NLP')
p_values_table['Model B'] = p_values_table['Model B'].str.replace('on GNN Data', 'on GNN Data - Fin + NLP')
# Repalce 'Pretrained on' with 'Pretrained, on'
p_values_table['Model A'] = p_values_table['Model A'].str.replace('Pretrained on', 'Pretrained, on')
p_values_table['Model B'] = p_values_table['Model B'].str.replace('Pretrained on', 'Pretrained, on')

# Export to LaTeX
# Center all columns
lt_string = p_values_table.to_latex(index=False, column_format='c' * 5, escape=False)
latex_with_font_size = "\\tiny\n" + lt_string + "\n\\normalsize"
with open('../../Output/Modelling/p_values_table.tex', 'w') as f:
    f.write(latex_with_font_size)

p_values_table

,Model A,Model B,"p, Non-Exact","p, Non-Exact, Continuity Corrected","p, Exact"
0,"Logistic - Exclude Previous Rating, Altman-Z Only",Rating Random Guess Model,0.06,0.07,0.07
1,Logistic - Exclude Previous Rating - Fin Only,Logistic - Exclude Previous Rating - Fin + NLP,0.31,0.35,0.35
2,Logistic - Include Previous Rating - Fin Only,Logistic - Include Previous Rating - Fin + NLP,-,-,-
3,Logistic - SMOTE Rating Change - Fin Only,Logistic - SMOTE Rating Change - Fin + NLP,0.17,0.19,0.19
4,XGBoost - Exclude Previous Rating - Fin Only,XGBoost - Exclude Previous Rating - Fin + NLP,0.0,0.0,0.0
5,XGBoost - Include Previous Rating - Fin Only,XGBoost - Include Previous Rating - Fin + NLP,-,-,-
6,XGBoost - SMOTE Rating Change - Fin Only,XGBoost - SMOTE Rating Change - Fin + NLP,0.06,0.11,0.11
7,XGBoost - SMOTE Rating Change - Fin Only,Change Random Guess Model,0.0,0.0,0.0
8,XGBoost - SMOTE Rating Change - Fin + NLP,Change Random Guess Model,0.0,0.0,0.0
9,GNN - Transductive - Fin + NLP,Logistic Regression - Retrain on GNN Data - Fi...,0.03,0.03,0.03
